# 6 - Functionalizing Ligands 

Beyond simple ligands searched from SMILES, we want to dig into functionalizations of ligands!

In this tutorial we will look at the ligand functionalization routines in Architector, covering:

**(A)** Viewing default functional groups present in architector by name!

**(B)** Identifying functionalization sites on ligands.

**(C)** Adding both single and multiple functionalizations to a single ligand.

In [ ]:
# First, import useful packages again
from architector import (build_complex, # Build routine
                         view_structures, # Visualization
                         smiles2Atoms, # Smiles to ASE Atoms
                         get_obmol_smiles, # Coversion to OBmol routine for editing ligand smiles
                         get_smiles_obmol, # Convert OBmol to SMILES string
                         convert_obmol_ase) # Conversion of OBmol molecele to ASE atoms for visualization
from architector.io_ptable import functional_groups_dict # Default functional groups dictionary
import copy

# For (A), Some Functional groups are included by default. Otherwise they can be input as SMILES strings!

Here, I will visualize all functional groups present by default in Architector. We will write a quick function for placing the functional groups onto a Benzene ring for clarity!

In [ ]:
def view_functional_group(name, base_group='c1ccccc1', base_inds=[5]):
    """
    function to visualize functional groups on a base_group. 
    
    Inputs:
    name : str
        Name of the functional group to put on the base_group.
    base_group : str, optional
        SMILES string of the organic group to functionalize! by default benzene.
    base_inds : list(Int) 
        list of the indices of the base_smiles to add the functional group to.
    """
    fgs = [{'functional_group':name,'smiles_inds':base_inds}] # Construct functional group dictionary list.
    OBmol = get_obmol_smiles(base_group, functionalizations=fgs) # Perform functionalization in Opebabel
    new_smiles = get_smiles_obmol(OBmol) # Get the new smiles for this OBmol molecule
    ase_atoms = convert_obmol_ase(OBmol) # Convert the molecule to ASE atoms for viewing
    # Print out the functional group name and edited SMILES string
    print('Name: {}\t Functionalized_Smiles: {}'.format(name,new_smiles))  
    view_structures(ase_atoms) # View the structures.

Now, we iterate through the default functional group dictionary, visualizing what each group looks like on benzene!

Note that after a certain number of in-notebook visualizations your computer's RAM may be taken up resulting in some 
of these structures not being visualized. Here, we cap the visualization at the first 10 functional groups. 

If you want to view another subset of the dictionary simply change the range (e.g. [0:10] -> [10:20])

In [ ]:
for key in list(functional_groups_dict.keys())[0:10]:
    view_functional_group(key)

# For (B), We can also functionalize ligands during complex construction! But we need to know where to functionalize them.

Fort this example we will be functionalizing bipyridine (bipy) when bound to Fe in a couple different ways to highlight this functionality.

To start, we need the base bipy SMILES and coordinating atoms:

In [ ]:
# From online:
bipy_smiles = 'n1ccccc1-c2ccccn2'
metal = 'Fe' # Initilize metal

Now, we can view the structures. 

### Instead of just looking for the coordinating atoms, we can also identify functionalization sites:

In [ ]:
bipy_atoms = smiles2Atoms(bipy_smiles)
view_structures(bipy_atoms,labelinds=True,w=500,h=500) 

As before, the coordination sites are the Nitrogens (blue) with indices 0 and 11, and that bipy will be a "bi_cis" ligand.

### However, if we want to put two functional groups at both of the positions "para" to the Nitrogens, these sites are the Carbons with indices 3 and 8!

In [ ]:
bipy_coordList = [0,11]
bipy_ligType = 'bi_cis'
para_smiles_inds = [3,8]

## For (C), Now we can make a functional group list of dictionaries for the bipy during complex construction! 

Here, let's just use the chloro functionalization from above for simplicity!

In [ ]:
para_functional_groups = [{'functional_group':'chloro','smiles_inds':para_smiles_inds}]

Now we have enough to do a functionalized complex construction with Fe!

In [ ]:
# We now have what we need to make an Fe-Bipy complex with functionalizations
lig_dict = {'smiles':bipy_smiles,
            'coordList':bipy_coordList,
            'ligType':bipy_ligType,}
            # 'functionalizations':para_functional_groups}

inputDict = {'core':{
    'metal':'Fe',
    'coreType':'octahedral'  # Just making octahedral complexes for simplicity
        },
    'ligands':[lig_dict], # Add in the ligands dictionary
    'parameters':{
        'assemble_method':'GFN-FF', # Switch to GFN-FF for faster assembly, 
        'relax':False, # Turn of relaxation for non-optimized structures
        'fill_ligand':0
                 }
    }

In [ ]:
out = build_complex(inputDict)

In [ ]:
view_structures(out,w=500,h=500)

Looks like a chloro-functionalized bipy at the para-positions!

### Beyond this, we can create more than one type of functionalization

Here, let's put Chloros at the para-positions and bromos at the ortho-positions to the coordinating atoms.

The ortho positions correspond to Carbon indices 2 and 9 above

In [ ]:
multi_fgs = [{'functional_group':'chloro','smiles_inds':[8,3]}, # Chloro at para positions
                     {'functional_group':'bromo','smiles_inds':[2,9]}] # Bromo at ortho-positions

new_inputDict = copy.deepcopy(inputDict) # Copy inputDict

new_inputDict['ligands'][0]['functionalizations'] = multi_fgs # Change bipy functionalization!
del new_inputDict['parameters']['fill_ligand'] # Remove fill ligand -> Default to H2O

In [ ]:
out1 = build_complex(new_inputDict)

Now, we have a water co-coordinated bi-functionalized Fe-Bipy complex!

In [ ]:
view_structures(out1,w=500,h=500)

Finally, we can make structures with both functionalized and un-functionalized ligands simultaneously.

Also, functional groups can be defined by SMILES strings. Here we introduce a butane functional group with a tail tricholor carbon end.

In [ ]:
functional_groups = [{'functional_group':'CCCC(Cl)(Cl)Cl','smiles_inds':[8,3]}]

lig_dict = {'smiles':bipy_smiles,'coordList':bipy_coordList,'ligType':bipy_ligType, # Functionalized bipy
            'functionalizations':functional_groups}
lig_dict2 = {'smiles':bipy_smiles,'coordList':bipy_coordList,'ligType':bipy_ligType} # Unfunctionalized bipy

last_inputDict = {
    'core':{
        'metal':'Fe',
        'coreType':'octahedral'
    },
    'ligands':[lig_dict,lig_dict2,lig_dict2],
    'parameters':{
        'assemble_method':'GFN-FF',
        'relax':False
    }
}
last_inputDict

In [ ]:
out = build_complex(last_inputDict)

In [ ]:
view_structures(out,w=500,h=500)

Looks great! Now we have covered functionalizations as well.

# Conclusions

In this tutorial we covered:

**(A)** Viewing default functional groups present in architector by name!

**(B)** Identifying functionalization sites on ligands.

**(C)** Adding both single and multiple functionalizations to a single ligand.